In [7]:
import tweepy
import pandas as pd
import mysql.connector
from dotenv import load_dotenv
import time 

In [8]:
# load data dari .env
load_dotenv(override=True)

# get detail mysql
db_host = os.getenv("mysql_host")
db_user = os.getenv("mysql_user")
db_password = os.getenv("mysql_password")
db_database = os.getenv("mysql_database")
db_port = os.getenv("mysql_port")

# get bearer token api twitter
bearer_token = os.getenv("bearer_token")

In [9]:
def insertTweet(db, tweet_id, data):
    
    mycursor = db.cursor()
    sql = "SELECT * FROM tweets WHERE tweet_id ="+ tweet_id
    mycursor.execute(sql)
    tweet = mycursor.fetchone()
    
    if not tweet:
        mycursor = db.cursor()
        sql = "INSERT INTO tweets (tweet_id, content, retweet_count, reply_count, like_count, source, next_pagination,  created_at) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
        val = (data['tweet_id'], data['content'], data['retweet_count'], data['reply_count'], data['like_count'], data['source'], data['next_pagination'],data['created_at'])
        mycursor.execute(sql, val)
        db.commit()
        mycursor.close()

In [10]:
# create client tweepy
client = tweepy.Client(bearer_token)

# create connection to database
db =  mysql.connector.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    database=db_database,
    port=db_port
)

In [12]:
keyword = "energi baru terbarukan -is:retweet"
search_df = pd.DataFrame()
loop = 1
loop_conditional = False
token = None

# initial search to get next_token or pagination
search = client.search_recent_tweets(
    query = keyword,
    tweet_fields=['public_metrics', 'author_id', 'created_at', 'source', 'text'], 
    max_results=50, 
    # next_token=token,
)
# save result of search to dataframe
for row in search.data:
    temp_data = pd.json_normalize(row.data ,  sep = "_")
    search_df = pd.concat([search_df, temp_data], ignore_index=True)  
    loop+=1

# check next_token exists
if ('next_token' in search.meta):
    token = search.meta['next_token']
    # loop forever
    while loop_conditional == False:
        # always check next_token
        if ('next_token' in search.meta):
            # search recent tweet with token
            search = None
            search = client.search_recent_tweets(
                query = keyword,
                tweet_fields=['public_metrics', 'author_id', 'created_at', 'source', 'text'], 
                max_results=50, 
                next_token=token,
            )

            for row in search.data:
                temp_data = pd.json_normalize(row.data ,  sep = "_")
                search_df = pd.concat([search_df, temp_data], ignore_index=True)    
                loop+=1

            # insert
            for index, row in search_df.iterrows():
                # save data tweet to new object
                new_array = {}
                new_array["tweet_id"] = row['id'] 
                new_array["author_id"] = row['author_id'] 
                new_array["source"] = row['source'] 
                new_array["content"] = row['text'] 
                new_array["retweet_count"] = row['public_metrics_retweet_count'] 
                new_array["reply_count"] = row['public_metrics_reply_count'] 
                new_array["like_count"] = row['public_metrics_like_count'] 
                new_array["next_pagination"] = row['public_metrics_quote_count'] 
                new_array["next_pagination"] = search.meta['next_token']
                new_array["created_at"] = row['created_at'] 
                insertTweet(db, str(new_array["tweet_id"]), new_array)
            
            print(loop)

            if loop > 250:
                time.sleep(60*15)
                loop = 1
        

101
151
201
251


KeyError: 'next_token'

In [13]:
search.meta

{'newest_id': '1541778995779231744',
 'oldest_id': '1541681062061166592',
 'result_count': 37}